In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import datetime
import re 
import requests
import sqlite3 
import shelve
import pandas as pd 
from collections import defaultdict
from dateutil.relativedelta import relativedelta
from itertools import chain
from bs4 import BeautifulSoup

from concurrent.futures import ProcessPoolExecutor
import concurrent.futures 

In [22]:
source_fields = ['Symbol', 'Company Name', 'Status', 'Share Price', 'Shares Offered', 'Shareholder Shares Offered', 'Shares Over Alloted', 'Offer Amount', 'Total Expenses', 'Lockup Period ', 'Lockup Expiration', 'Quiet Period Expiration', 'Underwriter', 'Company Counsel']
schema_fields = ['Symbol', 'Company Name', 'Priced', 'Share Price', 'Shares Offered', 'Shareholder Shares Offered', 'Shares Over Alloted', 'Offer Amount', 'Total Expenses', 'Lockup Period', 'Lockup Expiration', 'Quiet Period Expiration', 'Underwriter', 'Company Counsel']
camelc_fields = [''.join(x.split()) for x in schema_fields] 

def get_priced_spos(year, month, prefix='two_column_main_content_rptPricing_company'):
    print('Retrieving SPOs priced in %s-%s' % (year, month))
    source='http://www.nasdaq.com/markets/spos/activity.aspx?tab=pricings&month=%s-%s' % (year, month)
    soup = BeautifulSoup(requests.get(source).content, 'html.parser')
    return list({x.string: x.get('href') for x in soup.find_all('a', text=lambda x: x and x == x.upper())[1:] if x.get('id').startswith(prefix)}.values())

def parse_spo_filing(source, fields=lambda x: x in ['infoTable', 'tabpane3']):
    print('Retrieving %s' %(source))
    soup = BeautifulSoup(requests.get(source).content, 'html.parser')
    f = lambda x: 'Underwriter' if x == 'Lead Underwriter' else x
    return defaultdict(lambda: '--', {f(re.sub(r"\(.*\)", "", k)): v for k, v in chain.from_iterable([[y.get_text().split('\n')[1:-1] for y in reversed(x.find_all('tr'))] for x in soup.find_all('div', id=fields) ])})

def substitute_chars(x):
    if (x == '--'): 
        return None
    if (any(c.isalpha() for c in x)):
        return x
    else:
        return ''.join(e for e in x if e.isdigit() or e == '.')

def conform_schema(data, fields=source_fields, transform=lambda x: x[x.find('(')+1:x.find(')')]): 
    return(tuple([transform(data[x]) if x == 'Status' else substitute_chars(data[x]) for x in fields]))

def create_table(engine, table_name, fields, unique_fields, replace=False):
    if replace:
        engine.execute('DROP TABLE IF EXISTS {}'.format(table_name))
    return engine.execute('CREATE TABLE IF NOT EXISTS {}({}, CONSTRAINT unq UNIQUE ({}) ON CONFLICT REPLACE)'.format(table_name, ','.join(fields), ','.join(unique_fields)))

def parse(x):
    return conform_schema(parse_spo_filing(x))

def monthly_urls(t):
    return get_priced_spos(t[0], t[1]) 

def ingest_historical_data(start=None, end=datetime.date(datetime.date.today().year, datetime.date.today().month, 1)):
    unparameterized=tuple(['?' for x in camelc_fields])
    t = start
    times = [] 
    while t <= end:
        times += [(str(t.year), str(t.month).zfill(2))]
        t += relativedelta(months=1)
    with ProcessPoolExecutor(max_workers=16) as executor:
        future_results = {executor.submit(monthly_urls, time) for time in times}
        urls = [] 
        for future in concurrent.futures.as_completed(future_results):
            urls += future.result() 
    with ProcessPoolExecutor(max_workers=16) as executor:
        future_results = {executor.submit(parse, url): url for url in urls}
        rows = []
        for future in concurrent.futures.as_completed(future_results):
            rows.append(future.result())
    print('Ingesting scraped data from %s' %(start.year))
    return(rows)
        
def ingest_current_data(engine, table_name, current=datetime.date(datetime.date.today().year, datetime.date.today().month, 1)):
    unparameterized=tuple(['?' for x in camelc_fields])
    spo_data = [conform_schema(parse_spo_filing(x)) for x in get_priced_spos(str(today.year), str(today.month).zfill(2)).values()]
    print('Updating the latest data...')
    engine.executemany('INSERT INTO {} VALUES ({})'.format(table_name, ','.join(unparameterized)), spo_data)
    
def ingest_data_chunks(engine, table_name):
    start = datetime.date(2000, 1, 1)
    end = datetime.date(2018, 3, 1)
    year = start 
    while year <= end:
        pd.DataFrame(ingest_historical_data(year, datetime.date(year.year, year.month+11, year.day)), columns=camelc_fields).to_sql(table_name, engine, if_exists='append')
        year += relativedelta(years=1)
        

In [23]:
engine = sqlite3.connect('nasdaq.db')


In [26]:
ingest_data_chunks(engine, 'seasoned_offerings')

Retrieving SPOs priced in 2000-01
Retrieving SPOs priced in 2000-02
Retrieving SPOs priced in 2000-05
Retrieving SPOs priced in 2000-06
Retrieving SPOs priced in 2000-03
Retrieving SPOs priced in 2000-04
Retrieving SPOs priced in 2000-09
Retrieving SPOs priced in 2000-08
Retrieving SPOs priced in 2000-07
Retrieving SPOs priced in 2000-11
Retrieving SPOs priced in 2000-10
Retrieving SPOs priced in 2000-12
Retrieving https://www.nasdaq.com/markets/spos/company/three-five-systems-inc-5063-25953
Retrieving https://www.nasdaq.com/markets/spos/company/evercel-inc-1462-25519
Retrieving https://www.nasdaq.com/markets/spos/company/trico-marine-services-inc-11550-3385
Retrieving https://www.nasdaq.com/markets/spos/company/molecular-devices-corp-1322-24815
Retrieving https://www.nasdaq.com/markets/spos/company/kensey-nash-corp-10273-25797
Retrieving https://www.nasdaq.com/markets/spos/company/north-bay-bancorpca-71221-3129
Retrieving https://www.nasdaq.com/markets/spos/company/mgi-pharma-inc-8376

Retrieving https://www.nasdaq.com/markets/spos/company/kemet-corp-1825-25641
Retrieving https://www.nasdaq.com/markets/spos/company/cree-inc-1713-25828
Retrieving https://www.nasdaq.com/markets/spos/company/applied-micro-circuits-corp-9399-26145
Retrieving https://www.nasdaq.com/markets/spos/company/nuance-communications-71700-6299
Retrieving https://www.nasdaq.com/markets/spos/company/pdl-biopharma-inc-11207-7933
Retrieving https://www.nasdaq.com/markets/spos/company/boston-private-financial-holdings-inc-7688-8040
Retrieving https://www.nasdaq.com/markets/spos/company/neopharm-inc-3948-8001
Retrieving https://www.nasdaq.com/markets/spos/company/adstar-inc-65834-5850
Retrieving https://www.nasdaq.com/markets/spos/company/atmel-corp-3250-27369
Retrieving https://www.nasdaq.com/markets/spos/company/northern-star-financial-inc-11154-7033
Retrieving https://www.nasdaq.com/markets/spos/company/cti-biopharma-corp-5460-27185
Retrieving https://www.nasdaq.com/markets/spos/company/elantec-semic

Retrieving https://www.nasdaq.com/markets/spos/company/medarex-inc-1106-25623
Retrieving https://www.nasdaq.com/markets/spos/company/mattson-technology-inc-2366-24796
Retrieving https://www.nasdaq.com/markets/spos/company/live-microsystems-inc-11483-25269
Retrieving https://www.nasdaq.com/markets/spos/company/urban-one-inc-10611-4512
Retrieving https://www.nasdaq.com/markets/spos/company/whitehall-jewellers-inc-2173-25792
Retrieving https://www.nasdaq.com/markets/spos/company/applied-biosystems-inc-4837-25066
Retrieving https://www.nasdaq.com/markets/spos/company/american-superconductor-corp-de-11268-25267
Retrieving https://www.nasdaq.com/markets/spos/company/abiomed-inc-4093-25074
Retrieving https://www.nasdaq.com/markets/spos/company/futurelink-corp-2561-3314
Retrieving https://www.nasdaq.com/markets/spos/company/sooner-holdings-inc-158-3384
Retrieving https://www.nasdaq.com/markets/spos/company/western-digital-corp-10781-3748
Retrieving https://www.nasdaq.com/markets/spos/company/v

Retrieving https://www.nasdaq.com/markets/spos/company/hologic-inc-45-15932
Retrieving https://www.nasdaq.com/markets/spos/company/biomarin-pharmaceutical-inc-7265-15859
Retrieving https://www.nasdaq.com/markets/spos/company/armor-holdings-inc-11955-15827
Retrieving https://www.nasdaq.com/markets/spos/company/neurocrine-biosciences-inc-5102-15719
Retrieving https://www.nasdaq.com/markets/spos/company/allegheny-energy-inc-11559-12760
Retrieving https://www.nasdaq.com/markets/spos/company/valspar-corp-2224-12698
Retrieving https://www.nasdaq.com/markets/spos/company/pepsico-inc-5110-12747
Retrieving https://www.nasdaq.com/markets/spos/company/amsurg-corp-8302-12727
Retrieving https://www.nasdaq.com/markets/spos/company/o-charleys-inc-3764-12728
Retrieving https://www.nasdaq.com/markets/spos/company/mabvax-therapeutics-holdings-inc-74836-15050
Retrieving https://www.nasdaq.com/markets/spos/company/freds-inc-7690-15011
Retrieving https://www.nasdaq.com/markets/spos/company/american-communi

Retrieving https://www.nasdaq.com/markets/spos/company/amv-liquidating-trust-11773-13999
Retrieving https://www.nasdaq.com/markets/spos/company/doral-financial-corp-11011-12481
Retrieving https://www.nasdaq.com/markets/spos/company/integer-holdings-corp-76381-13914
Retrieving https://www.nasdaq.com/markets/spos/company/l3-communications-titan-corp-7047-14325
Retrieving https://www.nasdaq.com/markets/spos/company/graftech-international-ltd-11788-14329
Retrieving https://www.nasdaq.com/markets/spos/company/skillsoft-corp-67345-14313
Retrieving https://www.nasdaq.com/markets/spos/company/warner-chilcott-plc-82554-14053
Retrieving https://www.nasdaq.com/markets/spos/company/sl-green-realty-corp-8495-14291
Retrieving https://www.nasdaq.com/markets/spos/company/plug-power-inc-11641-14257
Retrieving https://www.nasdaq.com/markets/spos/company/charles-river-laboratories-international-inc-69864-14264
Retrieving https://www.nasdaq.com/markets/spos/company/gmx-resources-inc-80682-13947
Retrieving

...

Retrieving https://www.nasdaq.com/markets/spos/company/neos-therapeutics-inc-807805-81394
Retrieving https://www.nasdaq.com/markets/spos/company/smart-sand-inc-864559-82594
Retrieving https://www.nasdaq.com/markets/spos/company/synergy-pharmaceuticals-inc-694130-78884
Retrieving https://www.nasdaq.com/markets/spos/company/civeo-corp-922153-81377
Retrieving https://www.nasdaq.com/markets/spos/company/acasti-pharma-inc-786885-84889
Retrieving https://www.nasdaq.com/markets/spos/company/dolphin-entertainment-inc-625677-84071
Retrieving https://www.nasdaq.com/markets/spos/company/catalyst-biosciences-inc-187177-80544
Retrieving https://www.nasdaq.com/markets/spos/company/my-size-inc-389480-85423
Retrieving https://www.nasdaq.com/markets/spos/company/akers-biosciences-inc-668645-85426
Retrieving https://www.nasdaq.com/markets/spos/company/yield10-bioscience-inc-109383-85163
Retrieving https://www.nasdaq.com/markets/spos/company/city-office-reit-inc-923868-83864
Retrieving https://www.nasdaq

Retrieving https://www.nasdaq.com/markets/spos/company/getty-realty-corp-md-3836-77210
Retrieving https://www.nasdaq.com/markets/spos/company/air-industries-group-71457-83560
Retrieving https://www.nasdaq.com/markets/spos/company/shiloh-industries-inc-7399-83047
Retrieving https://www.nasdaq.com/markets/spos/company/egalet-corp-917959-80286
Retrieving https://www.nasdaq.com/markets/spos/company/yield10-bioscience-inc-109383-83259
Retrieving https://www.nasdaq.com/markets/spos/company/atossa-genetics-inc-824892-82805
Retrieving https://www.nasdaq.com/markets/spos/company/cara-therapeutics-inc-693401-83084
Retrieving https://www.nasdaq.com/markets/spos/company/blueprint-medicines-corp-924997-83048
Retrieving https://www.nasdaq.com/markets/spos/company/cryoport-inc-134700-82677
Retrieving https://www.nasdaq.com/markets/spos/company/condor-hospitality-trust-inc-5881-81479
Retrieving https://www.nasdaq.com/markets/spos/company/medigus-ltd-943888-82865
Retrieving https://www.nasdaq.com/marke

Retrieving https://www.nasdaq.com/markets/spos/company/malibu-boats-inc-922316-78358
Retrieving https://www.nasdaq.com/markets/spos/company/myokardia-inc-885210-82537
Retrieving https://www.nasdaq.com/markets/spos/company/healthcare-realty-trust-inc-2463-82844
Retrieving https://www.nasdaq.com/markets/spos/company/rosetta-genomics-ltd-719912-83634
Retrieving https://www.nasdaq.com/markets/spos/company/chegg-inc-711152-84319
Retrieving https://www.nasdaq.com/markets/spos/company/citius-pharmaceuticals-inc-841996-83700
Retrieving https://www.nasdaq.com/markets/spos/company/archrock-partners-lp-713372-81784
Retrieving https://www.nasdaq.com/markets/spos/company/nanostring-technologies-inc-746391-76335
Retrieving https://www.nasdaq.com/markets/spos/company/senseonics-holdings-inc-942412-83295
Retrieving https://www.nasdaq.com/markets/spos/company/asure-software-inc-9175-82832
Retrieving https://www.nasdaq.com/markets/spos/company/camping-world-holdings-inc-996615-83243
Retrieving https://w

Retrieving https://www.nasdaq.com/markets/spos/company/flexion-therapeutics-inc-763477-84097
Retrieving https://www.nasdaq.com/markets/spos/company/globalstar-inc-715019-84916
Retrieving https://www.nasdaq.com/markets/spos/company/22nd-century-group-inc-694416-82506
Retrieving https://www.nasdaq.com/markets/spos/company/us-gold-corp-4226-83669
Retrieving https://www.nasdaq.com/markets/spos/company/caredx-inc-394296-84649
Retrieving https://www.nasdaq.com/markets/spos/company/immunogen-inc-7817-82986
Retrieving https://www.nasdaq.com/markets/spos/company/newlink-genetics-corp-97316-78812
Retrieving https://www.nasdaq.com/markets/spos/company/paramount-gold-nevada-corp-954673-83822
Retrieving https://www.nasdaq.com/markets/spos/company/aethlon-medical-inc-9242-84317
Retrieving https://www.nasdaq.com/markets/spos/company/jaguar-health-inc-914820-84611
Retrieving https://www.nasdaq.com/markets/spos/company/ishares-silver-trust-677366-83223
Retrieving https://www.nasdaq.com/markets/spos/com

Retrieving https://www.nasdaq.com/markets/spos/company/sonoma-pharmaceuticals-inc-714052-85286
Retrieving https://www.nasdaq.com/markets/spos/company/aytu-bioscience-inc-731244-86020
Retrieving https://www.nasdaq.com/markets/spos/company/hamilton-lane-inc-1014270-86150
Retrieving https://www.nasdaq.com/markets/spos/company/checkpoint-therapeutics-inc-974640-85296
Retrieving https://www.nasdaq.com/markets/spos/company/g1-therapeutics-inc-891299-86215
Retrieving https://www.nasdaq.com/markets/spos/company/guidewire-software-inc-863116-86234
Retrieving https://www.nasdaq.com/markets/spos/company/sonoma-pharmaceuticals-inc-714052-85286
Retrieving https://www.nasdaq.com/markets/spos/company/aytu-bioscience-inc-731244-86020
Retrieving https://www.nasdaq.com/markets/spos/company/hamilton-lane-inc-1014270-86150
Retrieving https://www.nasdaq.com/markets/spos/company/checkpoint-therapeutics-inc-974640-85296
Retrieving https://www.nasdaq.com/markets/spos/company/g1-therapeutics-inc-891299-86215
R

In [27]:
pd.read_sql('select * from seasoned_offerings', engine)

,index,Symbol,CompanyName,Priced,SharePrice,SharesOffered,ShareholderSharesOffered,SharesOverAlloted,OfferAmount,TotalExpenses,LockupPeriod,LockupExpiration,QuietPeriodExpiration,Underwriter,CompanyCounsel
0,0,MOGN,MGI PHARMA INC,5/19/2000,18.00,1000000,,0,18000000.00,453000.00,180,11152000,6132000,U.S. Bancorp Piper Jaffray Inc,Dorsey & Whitney LLP
1,1,TMAR,TRICO MARINE SERVICES INC,5/25/2000,9.00,4500000,,,40500000.00,350000.00,180,11212000,6192000,Johnson Rice and Co. L.L.C,"Jones, Walker, Waechter, Poitevent, Carrer..."
2,2,ARXX,AEROFLEX INC,5/12/2000,29.00,3250000,750000,0,94250000.00,500000.00,180,1182000,662000,CIBC World Markets Corp,"Blau, Kramer, Wactlar and Lieberman, P.C"
3,3,MFLO,MOLDFLOW CORP,12/13/2000,19.00,2305000,1730000,0,43795000.00,500000.00,180,6112001,182001,"Adams, Harkness and Hill, Inc","Goodwin, Procter and Hoar llp"
4,4,VSH,VISHAY INTERTECHNOLOGY INC,5/10/2000,73.50,5800000,300000,,426300000.00,1250000.00,180,1162000,652000,"Bear, Stearns and Co. Inc",Kramer Levin Naftalis and Frankel LLP
5,5,CHRT,GLOBALFOUNDRIES SINGAPORE PTE. LTD.,5/5/2000,65.00,13510000,5710000,0,878150000.00,1800000.00,180,1112000,5302000,Credit Suisse First Boston Corp,Latham and Watkins
6,6,MDCC,MOLECULAR DEVICES CORP,5/25/2000,38.50,1500000,,,57750000.00,850000.00,180,11212000,6192000,ING Barings LLC,Cooley Godward LLP
7,7,TTPA,TRINTECH GROUP PLC,5/8/2000,22.56,6000000,2000000,0,135360000.00,2992247.00,180,1162000,622000,Deutsche Bank Securities Inc,"Wilson Sonsini Goodrich & Rosati, P.C."
8,8,ATO,ATMOS ENERGY CORP,12/15/2000,22.25,6000000,,,133500000.00,497000.00,180,6132001,192001,BofA Merrill Lynch,Gibson Dunn and Crutcher LLP
9,9,EVRC,EVERCEL INC,5/11/2000,12.50,1250000,0,0,15625000.00,500000.00,180,1172000,652000,Burnham Securities Inc,Robinson and Cole LLP
